In [12]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#pip install tensorflow and tensorflow_hub before running

In [14]:
#We will use these data
data=pd.read_csv('outfit_combinations.csv')
d3=pd.read_csv('Full+data.csv')
d3=d3.dropna(axis=1,how='all')
data=data.merge(d3[['product_id','description']],on='product_id',how='inner')
data=data.dropna()
data=data.reset_index().drop(columns='index')

In [15]:
# Run this cell after running the function cell and enter inputs either as product ID or Descriptions
#to get recommendations
#If you enter Product Id and it isn't found in our system, you'll see the closest product id
#You can enter either the query in format A or B (product id/description)
query=input("Please Enter your input \n")
recommend_USE(data,query)

Please Enter your input 
'shoe:(01DMBRYVA2ZFDYRYY5TRQZJTBE)'


,product_id,outfit_type,recommended_item
0,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,slim knit skirt
0,01DMBRYVA2PEPWFTT7RMP5AA1T,top,rib mock neck tank
0,01DMBRYVA2S5T9W793F4CY41HE,accessory1,medium margaux leather satchel
0,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,penelope mid cap toe pump


In [6]:
#Preprocess data - columns outfit_item_type,brand,product_full_name
def recommend_USE(data,query):
    from nltk.corpus import stopwords
    columns=['outfit_item_type','brand','product_full_name','description']
    for j in columns:
        for i in range(0,len(data)):
            new=re.sub('[^a-zA-Z0-9]', ' ',str(data.loc[i,j]))
            p=[]
            for k in new.split():
                separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
                new = separated.lower()
                p.append(new)
            ls=WordNetLemmatizer()
            new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
            new = ' '.join(new)
            data.loc[i,j]=new
    data['all_info']=data['product_id']+' '+data['brand']+' '+data['outfit_item_type']+' '+data['product_full_name']+" "+data['description']
    #Universal Sentence encoder Vectorize four columns
    import tensorflow as tf
    import tensorflow_hub as hub
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    V1 = embed(data['all_info'])
    #Preprocess data - columns outfit_item_type,brand,product_full_name
    new=re.sub('[^0-9a-zA-Z]', ' ',str(query))
    p=[]
    for k in new.split():
        separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
        new = separated.lower()
        p.append(new)
    ls=WordNetLemmatizer()
    new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
    new = ' '.join(new)
    query=new

    #Universal Sentence Vectorize Query
    Q = embed([query])
    from sklearn.metrics.pairwise import cosine_similarity
    #Find similarity with all documents
    results=cosine_similarity(V1,Q)
    
    import heapq
    k=list(data.iloc[heapq.nlargest(1,range(len(results)),results.__getitem__),0])
    for i in k:
        f=pd.DataFrame()
        for j in range(0,len(data)):
            l=[]
            if data.loc[j,'outfit_id']==i:
                l.append(list(data.iloc[j,[1,2,4]].values))
                if l!=[]:
                    f=f.append(l)
        f.columns=['product_id','outfit_type','recommended_item']
        f['product_id']=f['product_id'].str.upper()
        return f